# Fit the parameter recovery period for national data

In [11]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution
from datetime import datetime

from sirds_model import get_bounds_and_arguments, sirds_objective_function

## Reading data

In [12]:
df = pd.read_csv('data/output/df_ts_epidemic_episodes.csv', index_col=0)
df.DATA = pd.to_datetime(df.DATA)

## Preparing data

In [13]:
df = df.sort_values(by=['DATA'])

In [14]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df['CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL']/df['POPULACAO_2022'] * 100000 

In [15]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'].fillna(0)

In [16]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'].replace([np.inf, -np.inf], 0)

## Forecasting

In [17]:
# Define L2 regularization term
def l2_regularization(x, alpha):
    # L2 regularization term (sum of squared parameter values)
    return 0.5 * alpha * np.sum(x**2)

# Define the combined objective function with L2 regularization
def regularized_objective_function(x, *args):
    alpha=args[-1]    
    return sirds_objective_function(x, *args[:-1]) + l2_regularization(x, alpha)

In [ ]:
try:
    df_initial_results = pd.read_csv('data/estimation_results.csv')
except:
    df_initial_results = pd.DataFrame({'outbreak':[], 'alpha': []})

df_results = df_initial_results.copy()

FORECAST_HORIZON_IN_DAYS = 90
DAYS_TO_FIT_WITHIN_OUTBREAK = 21
DAYS_TO_RECOVERY = 8

DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR = 5
NUMBER_ESTIMATION_PER_REGULARIZATION_WEIGHT = 20

# Dates about begin time series and first case
date_first_case = min(df[(df.NOVOS_CASOS_SRAG > 0)].iloc[0].DATA, df[(df.CASOS_NOVOS_PAINEL > 0)].iloc[0].DATA)

# Define a range of alpha values to test
# alphas = [0, 0.001, 0.01, 0.1, 1, 10]
alphas = [0]

for outbreak in df['ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA'].dropna().unique():
    row = df[df.ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA == outbreak].iloc[0]
    print('Outbreak: ' + str(outbreak))
    outbreak_start_date = row['DATA']
    max_date_to_fit = outbreak_start_date + pd.DateOffset(days=DAYS_TO_FIT_WITHIN_OUTBREAK)
    
    # Period of analysis
    period_in_days = (max_date_to_fit - date_first_case).days + 1 + FORECAST_HORIZON_IN_DAYS
    max_date_to_analyze = date_first_case + pd.DateOffset(days=period_in_days)
                 
    for alpha in alphas:
        print('alpha: '+str(alpha))

        bounds, args = get_bounds_and_arguments(df, 'DATA', 'TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS_SIM', 'NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIA', 'TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL', 'ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA', DAYS_TO_RECOVERY, date_first_case, max_date_to_fit, df.POPULACAO_2022.iloc[0], period_in_days)
        args.append(alpha)
                                                      
        for estimation in range(NUMBER_ESTIMATION_PER_REGULARIZATION_WEIGHT):
            estimations_performed = len(df_results[(df_results.outbreak == outbreak) & (df_results.alpha == alpha)])
        
            if (estimation == estimations_performed):
                print('estimation: '+str(estimation))
                
                # Record the start time
                start_time = datetime.now()
                print(start_time)
        
                result = differential_evolution(regularized_objective_function, bounds, args=args, popsize=DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR, maxiter=10000, workers=3, updating='deferred')
        
                # Record the end time
                end_time = datetime.now()
        
                # Calculate the duration (in seconds) for the optimization
                duration = (end_time - start_time).total_seconds()
                print(duration)
                
                list_breakpoints_in_slow_transition = args[4]
                quantity_outbreaks = args[5]
                quantity_outbreak_adjustments = args[6]                
        
                # Create a dictionary to store results
                estimation_result = {
                    'outbreak': outbreak,
                    'alpha': alpha,
                    'estimation': estimation,  # To differentiate between multiple estimations
                    'result_fun': result.fun,
                    'result_nfev': result.nfev,
                    'result_nit': result.nit,
                    'result_success': result.success,
                    'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format start time as a string
                    'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format end time as a string
                    'duration_seconds': duration,  # Duration in seconds
                    'pop_size': DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR,
                    'period_in_days': period_in_days,
                    'days_to_recovery': DAYS_TO_RECOVERY,
                    'date_first_case': date_first_case,
                    'outbreak_start_date': outbreak_start_date,
                    'max_date_to_fit': max_date_to_fit,
                    'max_date_to_analyze': max_date_to_analyze,                    
                    'list_breakpoints_in_slow_transition': list_breakpoints_in_slow_transition,
                    'x_initial_infected_population': result.x[0],
                    'x_days_between_infections_0': result.x[1]         
                }
        
                quantity_epidemic_periods_with_slow_transition = len(list_breakpoints_in_slow_transition) + 1            
                
                for p in range(quantity_epidemic_periods_with_slow_transition):
                    estimation_result['x_case_fatality_probability_'+str(p)] = result.x[2 + p]
                    estimation_result['x_loss_immunity_in_days_'+str(p)] = result.x[2 + quantity_epidemic_periods_with_slow_transition + p]                                                 
                quantity_breakpoints = (quantity_outbreaks - 1) + (quantity_outbreak_adjustments)
                begin_breakpoint_parameters = 2 + 2*quantity_epidemic_periods_with_slow_transition
                for b in range(quantity_breakpoints):
                    estimation_result['x_days_between_infections_'+str(b+1)] = result.x[begin_breakpoint_parameters + b]
                    estimation_result['x_breakpoint_'+str(b+1)] = result.x[begin_breakpoint_parameters + quantity_breakpoints + b]
                    estimation_result['x_transition_days_between_epidemic_periods_'+str(b+1)] = result.x[begin_breakpoint_parameters + 2*quantity_breakpoints + b]
        
                print(estimation_result)
        
                # Append the estimation result to the list                
                df_results = pd.concat([df_results, pd.DataFrame.from_records([estimation_result])])
        
                df_results.to_csv('data/output/estimation_results.csv', index=False)           

Outbreak: 0.0
alpha: 0
estimation: 10
2023-11-23 08:14:18.532027
17.323638
{'outbreak': 0.0, 'alpha': 0, 'estimation': 10, 'result_fun': 0.3864851014381682, 'result_nfev': 1767, 'result_nit': 36, 'result_success': True, 'start_time': '2023-11-23 08:14:18', 'end_time': '2023-11-23 08:14:35', 'duration_seconds': 17.323638, 'pop_size': 5, 'period_in_days': 126, 'days_to_recovery': 8, 'date_first_case': Timestamp('2020-02-16 00:00:00'), 'outbreak_start_date': Timestamp('2020-03-01 00:00:00'), 'max_date_to_fit': Timestamp('2020-03-22 00:00:00'), 'max_date_to_analyze': Timestamp('2020-06-21 00:00:00'), 'list_breakpoints_in_slow_transition': [], 'x_initial_infected_population': 0.008171822342308278, 'x_days_between_infections_0': 3.1703052374538094, 'x_case_fatality_probability_0': 0.006003478141412302, 'x_loss_immunity_in_days_0': 131.61748862039008, 'x_days_between_infections_1': 3.190427032518366, 'x_breakpoint_1': 29.873205320137576, 'x_transition_days_between_epidemic_periods_1': 21.9691

In [ ]:
df_results.head()